# Table of contents 

1. [Introduction to the task](#1.-Introduction-to-the-task)

2. [Dataset](#2.-Datasets)
    
    2.1 [Datasets format](#2.1-Datasets-format)

3. [Model architecture](#3.-Model-architecture)

4. [Get started with the model](#4.-Get-started-with-the-model)
    
    4.1 [Installation](#4.1-Installation)

    4.2 [Support dataset configuration](#4.2-Support-dataset-configuration)

5. [Use the model for prediction](#5.-Use-the-model-for-prediction)

    5.1 [Predict using Python](#5.1-Predict-using-Python)
    
    5.2 [Predict using CLI](#5.2-Predict-using-CLI)
     
6. [Train the model on your data](#6.-Train-the-model-on-your-data)
    
    6.1 [Train your model from Python](#6.1-Train-your-model-from-Python)
    
    6.2 [Train your model from CLI](#6.2-Train-your-model-from-CLI)
    
7. [Evaluate](#7.-Evaluate)
    
    7.1 [Evaluate from Python](#7.1-Evaluate-from-Python)
    
    7.2 [Evaluate from CLI](#7.2-Evaluate-from-CLI)
    
8. [Metrics](#8.-Metrics)

# 1. Introduction to the task

__Text classification__ is a task of identifying one of the pre-defined label given an utterance, where label is one of N classes or "OOS" (out-of-scope examples - utterances that do not belong to any of the predefined classes). We consider few-shot setting, where only few examples (5 or 10) per intent class are given as a training set.


# 2. Dataset

In our experiments we used the [CLINC150](https://paperswithcode.com/dataset/clinc150) dataset, which has 10 different domains with 15 intents each, 100 shots per intent class and 1000 OOS examples. It simulates a setting, where model has to handle many different services with wide variety of intents.

Specifically, we validate our model on CLINC150 from the original DNNC paper. We parsed it to match the format described [below](#21-dataset-format). The original dataset can be downloaded from the DNNC [github page](https://github.com/salesforce/DNNC-few-shot-intent).

## 2.1 Datasets format

Train, dev and test set are separate json files, which have the following format

```
{
    "columns": [
        "text",
        "category"
    ],

    "data": [

        [
            "text"
            "label"
        ],

        ...
    ]
}
```

# 3. Model architecture

The typical methodology of few-shot text classification is to embed each example into a vector space and use an off-the-shelf distance metric to perform a similarity search. However, the text embedding methods do not discriminate the OOS examples well enough.


DNNC authors suggests to model fine-grained relations of utterance pairs via pairise simmilarity:

$h = BERT([[CLS], u, [SEP], e_{j,i}, [SEP]]) \in \R^d$

$S(u, e_{j,i}) = \sigma(W * h + b) \in \R$, where $e_{j, i} \in E $- training set, $W \in \R^{1×d}$, $b \in \R$

To mitigate the data scarcity setting in few-shot learning, DNNC uses knowldge-transfer from NLI task. We pretrain [roberta-base](https://huggingface.co/roberta-base) on combination of 3 NLI datasets: SNLI, WNLI, MNLI.

# 4. Get started with the model

## 4.1 Installation

First make sure you have the DeepPavlov Library installed.
[More info about the first installation.](http://docs.deeppavlov.ai/en/master/intro/installation.html)

In [ ]:
!pip install deeppavlov

Then make sure that all the required packages, datasets and weights are installed.

In [ ]:
!python -m deeppavlov install dnnc
!python -m deeppavlov download dnnc

`dnnc` is the name of the model's *config_file*. [What is a Config File?](http://docs.deeppavlov.ai/en/master/intro/configuration.html) 

Configuration file defines the model and describes it's hyperparameters

## 4.2 Support dataset configuration

Before making predictions or evaluation you need to set path to your support dataset. DNNC model compares input text to every example in support dataset to determine, which class the input example belongs to. By default, the model uses training set as support dataset. It is automatically saved by *dataset_iterator*  during the training step, but you can specify your own support dataset in the in `dnnc` config file. It has the same format as metioned [before]()

In [1]:
from deeppavlov.core.commands.utils import parse_config

model_config = parse_config('dnnc')

#  dataset for predictions
print(model_config['chainer']['pipe'][0]['support_dataset_path'])

~/.deeppavlov/parsed_datasets/parsed_dataset.json


### Off-the-shelf prediction

Base model was already pre-trained to recognize simmilar utterances, so you can use off-the-shelf model to make predictions and evalutation. No additional training needed.

### OOS prediction

Out-of-scope (OOS) examples are determined via confidence_threshold parameter with the following algorithm. Firstly model calculates an average similarity score for every class from support dataset. Secondly it determines the class with maximum similarity score. Finally the model predicts class with maximum similarity if it's score is higher than confidence_threshold and "oos" class otherwise. The higher the threshold, the more often the model predicts "oos" class. By default it is set to 0.5 and you can change it to your preferences in configuration file

In [3]:
from deeppavlov.core.commands.utils import parse_config

model_config = parse_config('dnnc')

#  dataset for predictions
print(model_config['chainer']['pipe'][-1]['confidence_threshold'])

0.5


# 5. Use the model for prediction

## 5.1 Predict using Python

After [installing](#4.-Get-started-with-the-model) the model, build it from the config and predict. If you set 'download' flag to 'True', then existing model weights will be overwritten. 

In [2]:
from deeppavlov import build_model, configs

model = build_model("dnnc", install=True, download=True)

model(["can you find me a good reviewed hotel in japan", "if i get a visa can i travel to japan"])

['book_hotel', 'international_visa']


## 5.2 Predict using CLI

You can also get predictions in an interactive mode through CLI.

In [ ]:
!python -m deeppavlov interact dnnc [-d] [-i]

Or make predictions for samples from *stdin*.

In [ ]:
!python -m deeppavlov predict dnnc [-d] [-i]

# 6. Train the model on your data

We have a separate `roberta_nli` config for training, which automatically transforms dataset into a pairwise format

To train the model on your data, you need to change the path to the dataset in `roberta_nli` config

In [4]:
from deeppavlov.core.commands.utils import parse_config

model_config = parse_config('roberta_nli')

#  dataset for training
print(model_config['dataset_reader']['data_path'])

~/.deeppavlov/downloads/clinc150_no_oos


## 6.1 Train your model from Python

In [ ]:
from deeppavlov import train_model

model = train_model("roberta_nli", install=True, download=True)

## 6.2 Train your model from CLI

In [ ]:
!python -m deeppavlov train roberta_nli [-d] [-i]

# 7. Evaluate

To evaluate the model on your data, you need to change the path to the dataset in `dnnc` config

In [5]:
from deeppavlov.core.commands.utils import parse_config

model_config = parse_config('dnnc')

#  dataset for evaluation
print(model_config['dataset_reader']['data_path'])

~/.deeppavlov/downloads/clinc150


## 7.1 Evaluate from Python

In [ ]:
from deeppavlov import evaluate_model

model = evaluate_model('dnnc', install=True, download=True)

## 7.2 Evaluate from CLI

In [ ]:
!python -m deeppavlov evaluate dnnc [-d] [-i]

# 8. Metrics

We follow the original DNNC paper approach and choose the best confidence_threshold on the dev set 

| Config name  | Dataset | Shot| Threshold |In-domain Accuracy | OOS Precision | OOS Recall | OOS F1 |
| :--- | --- | --- | --- | --- | --- | --- | ---: |
| dnnc | [CLINC150](https://paperswithcode.com/dataset/clinc150) | 5 | 0.6 | 88.8 | 71.0 | 86.2 | 77.8 |
| dnnc | [CLINC150](https://paperswithcode.com/dataset/clinc150) Banking Domain| 5 | 0.4 | 90.7 | 97.3 | 98.7 | 98.0 |
| dnnc | [CLINC150](https://paperswithcode.com/dataset/clinc150) Work Domain | 5 | 0.1 | 98.1 | 99.9 | 95.4 | 97.6 |